## Pré processamento das imagens

Retirado do kernel de the1owl (https://www.kaggle.com/the1owl/artificial-intelligence-for-cc-screening)

In [1]:
from PIL import ImageFilter, ImageStat, Image, ImageDraw
from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import glob
import cv2

In [6]:
def im_multi(path):
    try:
        im_stats_im_ = Image.open(path)
        return [path, {'size': im_stats_im_.size}]
    except:
        print(path)
        return [path, {'size': [0,0]}]

def im_stats(im_stats_df):
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
    for i in range(len(ret)):
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size'] = im_stats_df['path'].map(lambda x: ' '.join(str(s) for s in im_stats_d[x]['size']))
    return im_stats_df

def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (64, 64), cv2.INTER_LINEAR) #use cv2.resize(img, (64, 64), cv2.INTER_LINEAR)
    return [path, resized]

def normalize_image_features(paths):
    imf_d = {}
    p = Pool(cpu_count())
    ret = p.map(get_im_cv2, paths)
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    fdata = np.array(fdata, dtype=np.uint8)
    fdata = fdata.transpose((0, 3, 1, 2))
    fdata = fdata.astype('float32')
    fdata = fdata / 255
    return fdata

In [3]:
train = glob.glob('data/train/**/*.jpg') + glob.glob('data/additional/**/*.jpg')
train = pd.DataFrame([[p.split('/')[2],p.split('/')[3],p] for p in train], columns = ['type','image','path'])[::5]
print train

        type     image                             path
0     Type_3    38.jpg         data/train/Type_3/38.jpg
5     Type_3    58.jpg         data/train/Type_3/58.jpg
10    Type_3   431.jpg        data/train/Type_3/431.jpg
15    Type_3   194.jpg        data/train/Type_3/194.jpg
20    Type_3   480.jpg        data/train/Type_3/480.jpg
25    Type_3   435.jpg        data/train/Type_3/435.jpg
30    Type_3  1284.jpg       data/train/Type_3/1284.jpg
35    Type_3   253.jpg        data/train/Type_3/253.jpg
40    Type_3  1030.jpg       data/train/Type_3/1030.jpg
45    Type_3   324.jpg        data/train/Type_3/324.jpg
50    Type_3   585.jpg        data/train/Type_3/585.jpg
55    Type_3  1475.jpg       data/train/Type_3/1475.jpg
60    Type_3   914.jpg        data/train/Type_3/914.jpg
65    Type_3   530.jpg        data/train/Type_3/530.jpg
70    Type_3   636.jpg        data/train/Type_3/636.jpg
75    Type_3   468.jpg        data/train/Type_3/468.jpg
80    Type_3   307.jpg        data/train/Type_3/

In [7]:
train = im_stats(train)
train = train[train['size'] != '0 0'].reset_index(drop=True) #remove bad images
train_data = normalize_image_features(train['path'])
np.save('train.npy', train_data, allow_pickle=True, fix_imports=True)

le = LabelEncoder()
train_target = le.fit_transform(train['type'].values)
print(le.classes_) #in case not 1 to 3 order
np.save('train_target.npy', train_target, allow_pickle=True, fix_imports=True)

test = glob.glob('data/test/*.jpg')
test = pd.DataFrame([[p.split('/')[2],p] for p in test], columns = ['image','path'])
test_data = normalize_image_features(test['path'])
np.save('test.npy', test_data, allow_pickle=True, fix_imports=True)

test_id = test.image.values
np.save('test_id.npy', test_id, allow_pickle=True, fix_imports=True)

['Type_1' 'Type_2' 'Type_3']
